In [39]:
import os
import json
import pandas as pd 
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
key=os.getenv("OPENAI_API_KEY")

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
llm=ChatOpenAI(openai_api_key=key,model_name='gpt-3.5-turbo',temperature=0.6)

c:\Users\USER\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain

In [22]:
response_json={
    1:{"mcq":"question will be here",
     "options": {
            'a':"first option",
            'b':"second option",
            'c':"third option",
            'd':"fourth option"
        },
    
    "correct":"correct option"
    },
    2:{"mcq":"question will be here",
     "options": {
            'a':"first option",
            'b':"second option",
            'c':"third option",
            'd':"fourth option"
        },
    
    "correct":"correct option"
    },
}

In [23]:
template1="{text}.you have a text.now your task is to generate {number} mcq questions from this above text for {subject} student in {tone} tone.make sure that no mcq question is repeated and the questions are only from the above text.for generating mcq questions you have to follow the below response_json format.{response_json}"

In [24]:
quiz_generation_prompt1=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=template1
)

In [25]:
quiz=LLMChain(llm=llm,prompt=quiz_generation_prompt1,output_key="quiz")

In [26]:
template2="you are a good english grammarian and writer.now you have given a mcq question for {subject} student.your task is to analyze the question and generate a correct answer for the question {quiz}."

In [27]:
correct_answer_generation_prompt2=PromptTemplate(
    input_variables=["subject","quiz"],
    template=template2
)

In [28]:
answer=LLMChain(llm=llm,prompt=correct_answer_generation_prompt2,output_key="review")

In [29]:
mcq_chain=SequentialChain(chains=[quiz,answer],input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"])

In [30]:
path=r'C:\Users\USER\mcqgen\data.txt'
with open(path, 'r') as file:
    text=file.read()

In [37]:
mcq=mcq_chain({"text":text,"number":7,"subject":"Data Science","tone":"simple","response_json":response_json})
quiz_table=mcq['quiz']

In [48]:
df=pd.DataFrame(eval(quiz_table))
my_quiz_table=df.transpose()
print(my_quiz_table)

<class 'pandas.core.frame.DataFrame'>


In [49]:
my_quiz_table.to_csv("Data_Science_Quiz.csv")